In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# zip 파일 읽어오기
# 압축을 해제할 경로
%cd /content/drive/MyDrive/ai_data/240216
# 압축 파일 경로
!unzip -qq "/content/drive/MyDrive/ai_data/selected_images_0.1.zip"

/content/drive/MyDrive/ai_data/240216
warning [/content/drive/MyDrive/ai_data/selected_images_0.1.zip]:  2505018165 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [/content/drive/MyDrive/ai_data/selected_images_0.1.zip]:  start of central directory not found;
  zipfile corrupt.
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


In [ ]:
import os
import torch
import torchvision
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
sample_data_path = "/content/drive/MyDrive/ai_data/240216/selected_images_0.1/"
images_path = sample_data_path + "/images"
labels_path = sample_data_path + "/labels"
print(os.listdir(sample_data_path))
print(os.listdir(images_path))
print(os.listdir(labels_path))

['images', 'labels']
['EMOIMG_ANGER_SAMPLE', 'EMOIMG_PAIN_SAMPLE', 'EMOIMG_HAPPY_SAMPLE', 'EMOIMG_EMB_SAMPLE']
['EMOIMG_HAPPY_SAMPLE', 'EMOIMG_PAIN_SAMPLE', 'EMOIMG_EMB_SAMPLE', 'EMOIMG_ANGER_SAMPLE']


In [ ]:
print(images_path)
print(labels_path)

/content/drive/MyDrive/ai_data/240215/facial_expression_no_korean_samples/data/images
/content/drive/MyDrive/ai_data/240215/facial_expression_no_korean_samples/data/labels


### 라벨 내용 확인

In [ ]:
sample_label = os.listdir(labels_path + "/EMOIMG_ANGER_SAMPLE")[0]
print(sample_label)

d6119c82-2a17-4d1d-b233-d82866a2cc65.json


In [ ]:
import json
sample_label = os.listdir(labels_path + "/EMOIMG_ANGER_SAMPLE")[0]
with open(labels_path + "/EMOIMG_ANGER_SAMPLE/" + sample_label, "r", encoding = 'utf-8') as f:
    data = json.load(f)
print(data)

[{'filename': '15284a77b3979147794bb67ed52c6c308b6fb41ed8c813847f883e7f9a283fce_여_30_분노_스포츠 관람 및 레저시설_20201208114828-004-001.jpg', 'gender': '여', 'age': 30, 'isProf': '전문인', 'faceExp_uploader': '분노', 'bg_uploader': '스포츠 관람 및 레저시설', 'annot_A': {'boxes': {'maxX': 1138.7168, 'maxY': 877.3331000000001, 'minX': 713.65063, 'minY': 253.22485000000003}, 'faceExp': '중립', 'bg': '스포츠 관람 및 레저시설'}, 'annot_B': {'boxes': {'maxX': 1132.7748474934037, 'maxY': 853.5652899736147, 'minX': 707.7086774934037, 'minY': 229.45703997361477}, 'faceExp': '불안', 'bg': '스포츠 관람 및 레저시설'}, 'annot_C': {'boxes': {'maxX': 1138.7168, 'maxY': 877.3331000000001, 'minX': 713.65063, 'minY': 253.22485}, 'faceExp': '중립', 'bg': '스포츠 관람 및 레저시설'}}, {'filename': '15284a77b3979147794bb67ed52c6c308b6fb41ed8c813847f883e7f9a283fce_여_30_분노_오락&공연시설_20201208114841-005-004.jpg', 'gender': '여', 'age': 30, 'isProf': '전문인', 'faceExp_uploader': '분노', 'bg_uploader': '오락/공연시설', 'annot_A': {'boxes': {'maxX': 1092.5808002614367, 'maxY': 846.5450991

In [ ]:
len(data)

580

json 파일에서 이미지별 true 라벨 정보 빼기

In [ ]:
def get_image_counts(dirs, path): # 하위 폴더의 이미지 갯수 확인하기
  # get number of images in each sub-dir
  image_counts = {}
  for dir in dirs:
    image_counts[dir] = len(os.listdir(path + "/" + dir))
  return image_counts

In [ ]:
# 선들님이 해주시겠지만, 일단 야매로 해야함
def get_true_labels(image_counts, labels_dirs, labels_path):

  # # get number of images in each sub-dir
  # image_counts = {}
  # for dir in labels_dirs:
  #   image_counts[dir] = len(os.listdir(images_path + "/" + dir))

  # extract needed labels from json files in each sub-dir
  all_true_labels = {}
  for dir in labels_dirs:
    json_file =  os.listdir(labels_path  + "/" + dir )[0]
    with open(labels_path + "/" + dir + "/" + json_file, "r", encoding = 'utf-8') as f:
      data = json.load(f)
      true_labels = []
      for i in range(image_counts[dir]):
        true_labels.append({
            "filename": data[i]["filename"],
            "gender": data[i]["gender"],
            "age": data[i]["age"],
            "faceExp_uploader": data[i]["faceExp_uploader"],
            "boxes_annot_A": {"minX": data[i]["annot_A"]["boxes"]["minX"],
                              "minY": data[i]["annot_A"]["boxes"]["minY"],
                              "maxX": data[i]["annot_A"]["boxes"]["maxX"],
                              "maxY": data[i]["annot_A"]["boxes"]["maxY"],
                              "width": data[i]["annot_A"]["boxes"]["maxX"] - data[i]["annot_A"]["boxes"]["minX"],
                              "height": data[i]["annot_A"]["boxes"]["maxY"] - data[i]["annot_A"]["boxes"]["minY"]}
        })
    all_true_labels[dir] = true_labels

  return all_true_labels

In [ ]:
sample_data_path = "/content/drive/MyDrive/ai_data/240215/facial_expression_no_korean_samples/data"  # no korean ver
images_path = sample_data_path + "/images"
labels_path = sample_data_path + "/labels"
images_dirs = os.listdir(images_path) # list
labels_dirs = os.listdir(labels_path) # list

image_counts = get_image_counts(images_dirs, images_path)
all_true_labels = get_true_labels(image_counts, labels_dirs, labels_path)

In [ ]:
all_true_labels["EMOIMG_ANGER_SAMPLE"][1]

{'filename': '15284a77b3979147794bb67ed52c6c308b6fb41ed8c813847f883e7f9a283fce_여_30_분노_오락&공연시설_20201208114841-005-004.jpg',
 'gender': '여',
 'age': 30,
 'faceExp_uploader': '분노',
 'boxes_annot_A': {'minX': 676.4047589310203,
  'minY': 210.02791574801813,
  'maxX': 1092.5808002614367,
  'maxY': 846.5450991730407,
  'width': 416.1760413304164,
  'height': 636.5171834250226}}

### DeepFace face_extractor이용해서 detection + crop 진행
- yolo
- mediapipe

In [ ]:
%pip install ultralytics # for yolo
%pip install deepface
from deepface import DeepFace

24-02-16 00:51:49 - Directory /root/.deepface created
24-02-16 00:51:49 - Directory /root/.deepface/weights created


In [ ]:
sample_data_path = "/content/drive/MyDrive/ai_data/240216/selected_images_0.1"  # no korean ver
images_path = sample_data_path
images_dirs = os.listdir(images_path) # list
print(os.listdir(images_path))
# print(os.listdir(labels_path))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ai_data/240216/selected_images_0.1'

In [ ]:
image_counts = get_image_counts(images_dirs, images_path)

In [ ]:
image_counts

{'anger': 162,
 'anxiety': 161,
 'embarrass': 161,
 'happy': 160,
 'normal': 161,
 'pain': 160,
 'sad': 158}

In [ ]:
def get_pred_labels_and_crop(image_counts, images_dirs, images_path, detector):

  # new dirs for cropped image dataset
  root_path =  "/content/drive/MyDrive/ai_data/240216"
  save_dir = os.path.join(root_path, "cropped_selected_0.01")
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  # extract facial areas using DeepFace detection and crop images
  all_pred_labels = {}

  for dir in images_dirs:
    pred_labels = []
    # for i in range(image_counts[dir]):
    for i in range(image_counts[dir]):
      filename =  os.listdir(images_path  + "/" + dir)[i]
      img = plt.imread(os.path.join(images_path, dir, filename))
      img_array = np.array(img) # path로 넣는 경우에는 필요 없음

      # Use DeepFace extract_faces to detect & crop face
      face_objs = DeepFace.extract_faces(img_path = img_array,
                                         target_size = (224, 224),
                                         detector_backend = detector,
                                         enforce_detection = False # 얼굴이 감지 되지 않았을 때 뜨는 에러 avoid
                                         )
      # crop face and save in new dirs
      cropped_img_array = face_objs[0]['face']
      save_dir_path = os.path.join(save_dir, dir)
      if not os.path.exists(save_dir_path):
        os.makedirs(save_dir_path)
      plt.imsave(os.path.join(save_dir, dir, filename), cropped_img_array)

      # add predicted bbox
      pred_labels.append({
            "filename": filename,
            "boxes_pred": face_objs[0]['facial_area'] # ex: {'x': 1390, 'y': 745, 'w': 876, 'h': 1284}
        })
    all_pred_labels[dir] = pred_labels

  return all_pred_labels

In [ ]:
# T4, 4분 1초
all_pred_labels = get_pred_labels_and_crop(image_counts, images_dirs, images_path, "yolov8") # 'yolov8', 'mediapipe', 'mtcnn' 정도 확인

Downloading...
From: https://drive.google.com/uc?id=1qcr9DbgsX3ryrz2uU8w4Xm3cOrRywXqb
To: /root/.deepface/weights/yolov8n-face.pt
100%|██████████| 6.39M/6.39M [00:00<00:00, 87.2MB/s]


24-02-16 01:00:28 - Downloaded YOLO model yolov8n-face.pt


In [ ]:
len(all_pred_labels["sad"])

158

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터 폴더 경로
data_folder = "/content/drive/MyDrive/ai_data/240216/cropped_selected_0.01"

# 새로 저장할 데이터 폴더 경로
new_data_folder = "/content/drive/MyDrive/ai_data/240216/cropped_0.01_train_test"

# 감정 클래스 리스트
emotion_classes = ["anger", "happy", "pain", "sad", "normal", "anxiety", "embarrass"]
# emotion_mapper = {
#     "anger": "EMOIMG_ANGER_SAMPLE",
#     "happy": "EMOIMG_HAPPY_SAMPLE",
#     "pain": "EMOIMG_PAIN_SAMPLE",
#     "emb": "EMOIMG_EMB_SAMPLE"
# }

# train과 test 폴더 생성
for split_folder in ["train", "test"]:
    os.makedirs(os.path.join(new_data_folder, split_folder), exist_ok=True)
    for emotion in emotion_classes:
        os.makedirs(os.path.join(new_data_folder, split_folder, emotion), exist_ok=True)

# 각 클래스에 대해 train과 test로 이미지를 나누기
for emotion in emotion_classes:
    images_folder = os.path.join(data_folder, emotion)

    # train과 test 폴더 경로
    train_folder = os.path.join(new_data_folder, "train", emotion)
    test_folder = os.path.join(new_data_folder, "test", emotion)

    # 이미지 파일 목록 가져오기
    image_files = [f for f in os.listdir(images_folder) if f.endswith(".jpg")]  # 확장자에 따라 수정

    # train_test_split을 사용하여 이미지를 랜덤으로 나누기
    train_images, test_images = train_test_split(image_files, test_size=0.2, random_state=42)

    # train 폴더로 이미지 복사
    for train_image in train_images:
        shutil.copy(os.path.join(images_folder, train_image), os.path.join(train_folder, train_image))

    # test 폴더로 이미지 복사
    for test_image in test_images:
        shutil.copy(os.path.join(images_folder, test_image), os.path.join(test_folder, test_image))


DeepFace.extract_faces로 받아오는 bbox는 다음과 같다.
{'x': 1390, 'y': 745, 'w': 876, 'h': 1284}

여기서 x,y는 left-corner를 의미하므로
x = minX
y = maxY 가 된다. (COCO 포맷)

true labels은 현재 이런 형태이다.
```
{'filename': '15284a77b3979147794bb67ed52c6c308b6fb41ed8c813847f883e7f9a283fce_여_30_분노_스포츠 관람 및 레저시설_20201208114828-004-001.jpg',
 'gender': '여',
 'age': 30,
 'faceExp_uploader': '분노',
 'boxes_annot_A': {'minX': 713.65063,
  'minY': 253.22485000000003,
  'maxX': 1138.7168,
  'maxY': 877.3331000000001,
  'width': 425.06616999999994,
  'height': 624.10825}}
```

### 평가
-